In [ ]:
#cp ../../../downloads/roster_2023.parquet notebooks/roster_2023.parquet
#cp ../../../downloads/roster_2022.parquet notebooks/roster_2022.parquet
#cp ../../../downloads/roster_2021.parquet notebooks/roster_2021.parquet
#cp ../../../downloads/roster_2020.parquet notebooks/roster_2020.parquet
#cp ../../../downloads/roster_2019.parquet notebooks/roster_2019.parquet
#cp ../../../downloads/roster_2018.parquet notebooks/roster_2018.parquet
#cp ../../../downloads/player_stats_2017.parquet notebooks/player_stats_2017.parquet
#cp ../../../downloads/player_stats_2018.parquet notebooks/player_stats_2018.parquet
#cp ../../../downloads/player_stats_2019.parquet notebooks/player_stats_2019.parquet
#cp ../../../downloads/player_stats_2020.parquet notebooks/player_stats_2020.parquet
#cp ../../../downloads/player_stats_2021.parquet notebooks/player_stats_2021.parquet
#cp ../../../downloads/player_stats_2022.parquet notebooks/player_stats_2022.parquet
#cp ../../../downloads/"games.csv" notebooks/games.csv

In [7]:
import pandas as pd
import numpy as np

games = pd.read_csv('../data/games.csv')
pd.set_option("display.max_columns",55)

In [8]:
#target_season_num = 2022

def make_merge_frame(target_season_num):
    rosters = pd.read_parquet(f"../data/roster_{target_season_num}.parquet")
    stats = pd.read_parquet(f"../data/player_stats_{target_season_num-1}.parquet")
    rosters['player_display_name'] = rosters['full_name'] 
    target_season = games[games['season']==target_season_num].copy()#[(games['season']==2023)&(games['week']==1)]

    agg_stats = stats[stats['season_type']=='REG'].groupby(['player_display_name']).agg({
        'completions':'mean',
        'attempts':'mean',
        'passing_yards':'mean',
        'passing_tds':'mean',
        'interceptions':'mean',
        'sacks':'mean',
        'sack_yards':'mean',
        'sack_fumbles':'mean',
        'sack_fumbles_lost':'mean',
        'passing_air_yards':'mean',
        'passing_yards_after_catch':'mean',
        'passing_first_downs':'mean',
        'passing_epa':'mean',
        'passing_2pt_conversions':'mean',
        'pacr':'mean',
        'carries':'mean',
        'rushing_yards':'mean',
        'rushing_tds':'mean',
        'rushing_fumbles':'mean',
        'rushing_fumbles_lost':'mean',
        'rushing_first_downs':'mean',
        'rushing_epa':'mean',
        'rushing_2pt_conversions':'mean',
        'receptions':'mean',
        'targets':'mean',
        'receiving_yards':'mean',
        'receiving_tds':'mean',
        'receiving_fumbles':'mean',
        'receiving_fumbles_lost':'mean',
        'receiving_air_yards':'mean',
        'receiving_yards_after_catch':'mean',
        'receiving_first_downs':'mean',
        'receiving_epa':'mean',
        'receiving_2pt_conversions':'mean',
        'racr':'mean',
        'target_share':'mean',
        'air_yards_share':'mean',
        'wopr':'mean',
        'special_teams_tds':'mean',
        'fantasy_points':'mean',
        'fantasy_points_ppr':'mean',
        'week':'count'
    }).reset_index()
    agg_stats.rename({'week':'games'},axis=1,inplace=True)
    agg_stats = pd.DataFrame(np.where(agg_stats==0,np.nan,agg_stats),columns=agg_stats.columns)
    roster_stats = rosters.merge(agg_stats,on=['player_display_name'],how='inner')
    team_stats = roster_stats.groupby(['team']).agg({
        i:'sum' for i in agg_stats.columns if i not in ['player_display_name']
    }).reset_index()
    prep_frame = target_season[['week','away_team','home_team','away_score','home_score','result','spread_line']].copy()
    away_team_stats = team_stats.copy()
    home_team_stats = team_stats.copy()

    away_team_stats.columns = ['away_'+i for i in team_stats.columns]
    home_team_stats.columns = ['home_'+i for i in team_stats.columns]

    merge = prep_frame.merge(away_team_stats,on='away_team',how='left').merge(home_team_stats,on='home_team',how='left')

    merge['season'] = target_season_num
    return merge

In [9]:


merges = [make_merge_frame(target_season_num=i) for i in [2023,2022,2021,2020,2019,2018]]

In [10]:
all_dat = pd.concat(merges)

In [12]:
all_dat.to_csv("../data/model_data.csv",index=False)